In [ ]:
import os
import json

from lightning_whisper_mlx import LightningWhisperMLX
from tqdm import tqdm

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def write_json(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, ensure_ascii=False)

### 음성 데이터 변환

In [ ]:
model = "large-v3"

path_voice = "./data/voices"
list_voice = tqdm(os.listdir(path_voice))

whisper = LightningWhisperMLX(model=model, batch_size=12, quant=None)
result = []

for voice_file in list_voice:
    list_voice.set_description(voice_file)

    path_voice_file = os.path.join(path_voice, voice_file)
    translate = whisper.transcribe(audio_path=path_voice_file)

    resp = {"file_name": voice_file, "original": translate}

    write_json(resp, f"./data/output/{voice_file}.json")
    result.append(resp)

In [ ]:
import duckdb
import pandas as pd

df = duckdb.query("SELECT * FROM './data/output/*.json'").df()

df_file_name = df["file_name"]
df = pd.json_normalize(df["original"])
df = pd.concat([df_file_name, df], axis=1)
df

In [ ]:
df.loc[~df["language"].str.contains("ko")]

In [ ]:
# del df["segments"]

df.to_csv("./data/output.csv", index=False)

### 프롬프트 테스트

In [ ]:
template = """
<요청사항>
- 당신은 개인정보 보호를 위해 검토 요청을 받았습니다.
- 답변은 한글(korean)로 작성 부탁드립니다.
- <검토 내용>을 확인하고 개인정보라고 판단되는 내용을 마스킹합니다. 
- 마스킹은 별표(*)로 처리하고 개인정보를 지우셔야 됩니다.
- 마스킹 후 <검토 내용> 전문 전체를 마스킹이 된 답변으로 제공해야 합니다.
- 미스킹을 하지않고 답변을 낼 경우 개인정보 보호법에 위반될 수 있습니다.

<마스킹해야 하는 개인정보 목록>
- 이름, 주민등록번호, 연락처, 주소, 이메일주소, 계좌번호, 신용카드번호, 생년월일, 성별, 소속, 사업자등록번호

<검토 내용>
{input}
"""

In [ ]:
llm = ChatOllama(model="mistral")
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

In [ ]:
text = result[0]["original"]["text"]
print(chain.invoke({"input": text}))

In [ ]:
# gpt-4o로 생성된 텍스트
text = """
고객: 안녕하세요, 저는 부산 해운대구에 거주하는 이지은입니다. 최근에 귀사의 헬스케어 서비스를 이용하고 있는데, 몇 가지 문제가 있어서 문의드립니다. 서비스 이용을 시작한 날짜는 5월 20일이고, 가입 번호는 87654321입니다. 운동 기록과 관련된 몇 가지 사항을 수정하고 싶습니다. 제 전화번호는 010-5678-1234이고, 이메일은 jieun.lee@example.com입니다. 제 주민등록번호는 950505-2345678입니다. 문제가 해결되지 않으면 환불 요청도 고려하고 있습니다. 환불을 위한 계좌번호는 321-654-987654입니다.

상담원: 안녕하세요, 이지은 고객님. 저희 헬스케어 서비스를 이용해주셔서 감사합니다. 먼저 불편을 드려 죄송합니다. 말씀해주신 내용을 토대로 가입 번호 87654321를 확인해보겠습니다. 잠시만 기다려 주시겠습니까?

고객: 네, 기다리겠습니다.

상담원: 이지은 고객님, 가입 번호 87654321로 확인한 결과, 5월 20일에 헬스케어 서비스를 시작하신 내역이 맞습니다. 운동 기록 수정 관련 문의를 주셨는데요. 수정하고 싶은 항목을 구체적으로 말씀해주시면 도와드리겠습니다.

고객: 네, 최근 운동 기록이 잘못 입력된 것 같습니다. 6월 10일부터 6월 15일까지의 운동 시간이 실제와 다릅니다. 이 기간 동안 매일 1시간씩 운동했는데, 시스템에는 매일 30분만 운동한 것으로 기록되어 있습니다.

상담원: 알겠습니다, 이지은 고객님. 확인해본 결과 시스템 오류가 있었던 것 같습니다. 6월 10일부터 6월 15일까지의 운동 시간을 1시간으로 수정해드리겠습니다. 또한, 주민등록번호를 확인해주셔서 감사합니다. 환불 요청 시 필요한 정보로 저장해두겠습니다. 환불 절차를 안내드릴까요?

고객: 네, 환불 절차도 알고 싶습니다. 만약 문제가 지속되면 환불을 진행하고 싶습니다.

상담원: 환불 절차는 간단합니다. 환불 요청서를 작성해주시면, 계좌번호 321-654-987654로 3일 이내에 환불 처리됩니다. 이메일 jieun.lee@example.com로 환불 요청서 양식을 보내드리겠습니다. 추가로 궁금한 사항이 있으신가요?

고객: 지금은 없습니다. 빠른 문제 해결과 환불 요청서 안내 부탁드립니다.

상담원: 네, 이지은 고객님. 문제 해결과 환불 요청서 안내를 신속히 처리하겠습니다. 다른 문의 사항이 생기면 언제든지 연락주세요. 좋은 하루 되세요.

고객: 감사합니다. 좋은 하루 되세요.

"""

In [ ]:
print(chain.invoke({"text": text}))